In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

In [2]:
class SMA_Backtester():
    ''' Class for Vectorized Backtesting of SMA based trading Strategies
    '''
    def __init__(self, symbol, SMA_S, SMA_L, start, end):
        '''Parameters
        --------------
        symbol: str
            ticker symbol to be backtested
        SMA_S: int
            moving window in bars(eg: days) for shorter SMA
        SMA_L: int
            moving window in bars(eg: days) for longer SMA
        start: str
            start date for data import 
        end: str
            end date for data import
        
        '''
        self.symbol = symbol
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
        self.start = start
        self.end = end
        self.results = None
        self.get_data()
        self.prepare_data()
        
    def __repr__(self):
        return 'SMA_Back_Tester(symbol = {}, SMA_S = {}, SMA_L = {}, start = {}, end = {})'.format(self.symbol, self.SMA_S,
                                                                                                  self.SMA_L, self.start, self.end)
        
    def get_data(self):
        ''' Imports the data for 'forex_pairs.csv' (source can be changed)
        '''
        raw = pd.read_csv('forex_pairs.csv', parse_dates= ['Date'], index_col= 'Date')
        raw = raw[self.symbol].to_frame().dropna()
        raw = raw.loc[self.start: self.end].copy()
        raw.rename(columns= {self.symbol: 'price'}, inplace = True)
        raw['returns'] = np.log(raw/raw.shift(1))
        self.data = raw
        
    
    def prepare_data(self):
        ''' Prepares the data for strategy Backtesting
        '''
        data = self.data.copy()
        data['SMA_S'] = data.price.rolling(self.SMA_S).mean()
        data['SMA_L'] = data.price.rolling(self.SMA_L).mean()
        self.data = data
        
        
    def set_parameters(self, SMA_S = None, SMA_L = None):
        '''Updates SMA parameters and prepares dataset
        '''
        if SMA_S is not None:
            self.SMA_S = SMA_S
            self.data['SMA_S'] = self.data.price.rolling(self.SMA_S).mean()
        
        if SMA_L is not None:
            self.SMA_L = SMA_L
            self.data['SMA_L'] = self.data.price.rolling(self.SMA_L).mean()
    
    def test_strategy(self):
        '''Backtests the SMA based trading strategy
        '''
        data = self.data.copy().dropna()
        data['position'] = np.where(data['SMA_S'] > data['SMA_L'], 1, -1)
        data['r_strategy'] = data.position.shift(1) * data['returns']
        data.dropna(inplace = True)
        data['cumreturns'] = data['returns'].cumsum().apply(np.exp)
        data['cumstrategy'] = data['r_strategy'].cumsum().apply(np.exp)
        self.results = data
        
        performance = data.cumstrategy.iloc[-1] #absolute performance
        outperformace = data.cumstrategy.iloc[-1] - data.cumreturns.iloc[-1]
        return round(performance, 6), round(outperformace, 6)
    
    def plot_results(self):
        ''' Plots the performace of the trading strategy and compares to 'Buy and Hold'
        '''
        if self.results is None:
            print('Run test_strategy first')
        else:
            title = '{} | SMA_S{} | SMA_L{}'.format(self.symbol, self.SMA_S, self.SMA_L)
            self.results[['cumreturns', 'cumstrategy']].plot(title = title, figsize = (12,8))
        
    def optimize_parameters(self, SMA_S_range, SMA_L_range):
        ''' Finds the optimal strategy (global maximum) given the SMA parameter ranges.
            
            Parameters
            -----------
            SMA_S_range, SMA_L_range: tuple
            tuples of the form(start, end, step_size)    
        '''
        combinations = list(product(range(*SMA_S_range), range(*SMA_L_range)))
        
        #test all combinations
        results = []
        for comb in combinations:
            self.set_parameters(comb[0], comb[1])
            results.append(self.test_strategy()[0]) #only appending performance and not outperformance
            
        best_perf = np.max(results)  #best performance
        opt = combinations[np.argmax(results)] #optimal parameters.
        
        #run/set optimal parameters
        self.set_parameters(opt[0], opt[1])
        self.test_strategy()
        
        #creating a df with many results
        # created for further analysis.
        many_results = pd.DataFrame(data= combinations, columns = ['SMA_S', 'SMA_L'])
        many_results['performance'] = results
        self.results_overview = many_results
        
        return opt, best_perf

In [3]:
tester = SMA_Backtester('EURUSD=X', 50,200, '2004-01-01', '2020-06-30')

In [4]:
tester

SMA_Back_Tester(symbol = EURUSD=X, SMA_S = 50, SMA_L = 200, start = 2004-01-01, end = 2020-06-30)

In [5]:
tester.optimize_parameters((10, 50, 1), (100, 252, 1))

((46, 137), 2.526694)